# Introduction

Keyword-based search engines are easy to use and work well in most scenarios. However, they usually struggle with complex or long queries and words that have a dual meaning. Semantic (also called vector-based) search engines tackle those pitfalls by finding a numerical representation of text queries using state-of-the-art language models, indexing them in a high-dimensional vector space and measuring how similar a query vector is to the indexed documents.









We will build semantic search engines using cosine similarity to retrieve news based on a sentence. cosine similarity is the most common metric used to calculate the similarity between document text from input keywords/sentences. The model will match what we write with a news database and suggest the top five news. 

## Dataset: BBC
All rights, including copyright, in the content of the original articles are owned by the BBC.

- Consists of 2225 documents from the BBC news website corresponding to stories in five topical areas from 2004-2005.
- Class Labels: 5 (business, entertainment, politics, sport, tech)

Data Source: http://mlg.ucd.ie/datasets/bbc.html

# Load Data

In [1]:
# Import required libraries
import pandas as pd
import numpy as np

import re
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from gensim.models.word2vec import Word2Vec
from sentence_transformers import SentenceTransformer, util
import torch

In [2]:
# Load data set
df = pd.read_csv('BBC News Train.csv', usecols = ['ArticleId','Text'])
df1 = pd.read_csv('BBC News Test.csv', usecols = ['ArticleId','Text'])
newdf = pd.concat([df, df1], ignore_index=True)

print(newdf.shape)
newdf = newdf[newdf['Text'].notnull()]
print(newdf.shape)
newdf.head()

(2225, 2)
(2225, 2)


,ArticleId,Text
0,1833,worldcom ex-boss launches defence lawyers defe...
1,154,german business confidence slides german busin...
2,1101,bbc poll indicates economic gloom citizens in ...
3,1976,lifestyle governs mobile choice faster bett...
4,917,enron bosses in $168m payout eighteen former e...


# Text Preprocessing

- Get rid of stopwords such as 'a', 'the', 'is', etc. which are not informative first
- Make all the letters lowercase
- Remove any funky characters with a blank
- Remove inflectional endings to only return the base or dictionary form of a word (Lemmatization)
- Use the tokenizer to take a given sentence and parse it into a list with individual words separated by a comma

In [3]:
# let's remove some of the stop words
nltk.download('stopwords') # download stopwords

stopwords_eng = stopwords.words('english')
minlen = 4
maxlen = 500

def clean_text(text):
    ''' Pre process and convert texts to a list of words '''
    # convert to lowercase
    text = text.lower()
    
    # Clean the text
    text = re.sub(r"\\r\\n", " ", text)
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"-", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = text.strip()
    words = [word for word in text.split() if word not in stopwords_eng]
    text = " ".join(words)
    return text

def tokenizer(sentence, min_words=minlen, max_words=maxlen, stopwords=stopwords_eng, lemmatize=True):
    if lemmatize:
        stemmer = WordNetLemmatizer()
        tokens = [stemmer.lemmatize(w) for w in word_tokenize(sentence)]
    else:
        tokens = [w for w in word_tokenize(sentence)]
    token = [w for w in tokens if (len(w) > min_words and len(w) < max_words
                                                        and w not in stopwords)]
    return tokens 

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\newbm\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
# Check the results
newdf['clean_text'] = newdf['Text'].apply(clean_text)
newdf['token_text'] = newdf['clean_text'].apply(tokenizer)
newdf['token_text']

0       [worldcom, ex, bos, launch, defence, lawyer, d...
1       [german, business, confidence, slide, german, ...
2       [bbc, poll, indicates, economic, gloom, citize...
3       [lifestyle, governs, mobile, choice, faster, b...
4       [enron, boss, 168m, payout, eighteen, former, ...
                              ...                        
2220    [eu, probe, alitalia, state, aid, european, co...
2221    [u2, play, grammy, award, show, irish, rock, b...
2222    [sport, betting, rule, spotlight, group, mp, p...
2223    [alfa, romeo, get, gm, engine, fiat, stop, mak...
2224    [citizenship, event, 18, touted, citizenship, ...
Name: token_text, Length: 2225, dtype: object

# Semantic Search Engine

## TF-IDF

TF-IDF is a technique to calculate the weight of each word signifies the importance of the word in the document and corpus. This algorithm is mostly using for the retrieval of information and text mining field.

- TF-IDF = Term Frequency (TF) * Inverse Document Frequency (IDF)

In [5]:
# Fit the model
vectorizer = TfidfVectorizer(stop_words=stopwords_eng, tokenizer=tokenizer) 
tfidf_mat = vectorizer.fit_transform(newdf['clean_text'].values) # -> (num_sentences, num_vocabulary)
tfidf_mat.shape

C:\Users\newbm\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'re", "'s", "'ve", 'could', 'doe', 'ha', 'might', 'must', "n't", 'need', 'sha', 'wa', 'wo', 'would'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


(2225, 26258)

In [6]:
def search_tfidf(sentence):
    
    """
    Return the database sentences in order of highest cosine similarity relatively to each 
    token of the target sentence. 
    """
    # Embed the query sentence
    tokens = [str(tok) for tok in tokenizer(sentence)]
    vec = vectorizer.transform(tokens)
    # Create list with similarity between query and dataset
    mat = cosine_similarity(vec, tfidf_mat)
    
    cos_sim = np.mean(mat, axis=0)
    index = np.argsort(cos_sim)[::-1] # from highest idx to smallest score 
    mask = np.ones(len(cos_sim))
    mask = np.logical_or(cos_sim[index] != 0, mask) #eliminate 0 cosine distance
    best_index = index[mask][:5]  
    return display(newdf[['ArticleId', 'Text']].iloc[best_index])

## Word2Vec

Word embedding is one of the most popular representation of document vocabulary. It is capable of capturing context of a word in a document, semantic and syntactic similarity, relation with other words, etc. Word2Vec is one of the most popular technique to learn word embeddings using shallow neural network.

In [7]:
# Fit the model
dataset = newdf['token_text']
word2vec_model = Word2Vec(min_count=0, workers = 8, vector_size=300) 
word2vec_model.build_vocab(dataset.values)
word2vec_model.train(dataset.values, total_examples=word2vec_model.corpus_count, epochs=30)

(14870179, 15225510)

In [8]:
dataset = newdf['token_text']

def is_word_in_model(word, model):
    """
    Check on individual words ``word`` that it exists in ``model``.
    """
    assert type(model).__name__ == 'KeyedVectors'
    is_in_vocab = word in model.key_to_index.keys()
    return is_in_vocab

def search_w2v(query_sentence, topk=5):
    
    query_sentence = query_sentence.split()
    in_vocab_list, best_index = [], [0]*topk
    for w in query_sentence:
        # remove unseen words from query sentence
        if is_word_in_model(w, word2vec_model.wv):
            in_vocab_list.append(w)
    # Retrieve the similarity between two words as a distance
    if len(in_vocab_list) > 0:
        sim_mat = np.zeros(len(dataset))  # TO DO
        for i, data_sentence in enumerate(dataset):
            if data_sentence:
                sim_sentence = word2vec_model.wv.n_similarity(
                        in_vocab_list, data_sentence)
            else:
                sim_sentence = 0
            sim_mat[i] = np.array(sim_sentence)
        # Take the five highest norm
        best_index = np.argsort(sim_mat)[::-1][:topk]
    return display(newdf[['ArticleId', 'Text']].iloc[best_index])

## SentenceTransformers

SentenceTransformers is a Python framework for state-of-the-art sentence, text and image embeddings. We can use this framework to compute sentence / text embeddings for more than 100 languages. These embeddings can then be compared with cosine-similarity to find sentences with a similar meaning. This can be useful for semantic textual similar, semantic search, or paraphrase mining. Here, we will use a 'paraphrase-MiniLM-L6-v2' model which performs great in Semantic Textual Similarity tasks.

In [9]:
def search_transformers(query_sentence):
    model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
    corpus_embeddings = model.encode(newdf['clean_text'].values, convert_to_tensor=True)
    query_embedding = model.encode(query_sentence, convert_to_tensor=True)
    
    # We use cosine-similarity and torch.topk to find the highest 5 scores
    cos_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
    top_results = torch.topk(cos_scores, k=5)

    for score, idx in zip(top_results[0], top_results[1]):
        score = score.cpu().data.numpy() 
        idx = idx.cpu().data.numpy()
        print(newdf[['ArticleId', 'Text']].iloc[idx])  

# Search for news

In [10]:
query_sentence = "2004 Indian Ocean earthquake and tsunami" 

pd.options.display.max_colwidth = 500

In [11]:
# TF-IDF
search_tfidf(query_sentence)

,ArticleId,Text
1017,1790,us tv special for tsunami relief a us television network will screen a celebrity tv special to benefit the tsunami relief effort in south asia. nbc will encourage viewer donations during an hour-long show featuring musical performances on 15 january. actress sandra bullock has donated $1m (£525 000) to the american red cross and actor leonardo dicaprio pledged a sizable aid contribution to unicef. meanwhile 70 hong kong music and movie stars re-recorded we are the world in mandarin and ca...
2071,837,tsunami cost hits jakarta shares the stock market in jakarta has seen its biggest slide in a month after the country doubled the likely cost of rebuilding from the asian tsunami. the fall came as indonesia said it expected debt repayments of up to 30 trillion rupiah ($3.2bn; £1.7bn) to be frozen to help pay for the recovery. by monday s close the jakarta stock exchange was down 2.1% at 1 011.15. bar a slight dip at the new year the jse had risen steadily by 4.7% since the tsunami hit on ...
894,1677,gurkhas to help tsunami victims britain has offered to send a company of 120 gurkhas to assist with the tsunami relief effort in indonesia downing street said. the deployment would involve troops from the 2nd battalion royal gurkha rifles based in brunei. discussions have begun with indonesia on the exact timing and location of the deployment but the government said the offer was aimed at the aceh province. downing st said a similar offer might be made to the sri lankan government. howe...
2140,5,ocean s twelve raids box office ocean s twelve the crime caper sequel starring george clooney brad pitt and julia roberts has gone straight to number one in the us box office chart. it took $40.8m (£21m) in weekend ticket sales according to studio estimates. the sequel follows the master criminals as they try to pull off three major heists across europe. it knocked last week s number one national treasure into third place. wesley snipes blade: trinity was in second taking $16.1m (£8...
1391,1854,ocean s twelve raids box office ocean s twelve the crime caper sequel starring george clooney brad pitt and julia roberts has gone straight to number one in the us box office chart. it took $40.8m (£21m) in weekend ticket sales according to studio estimates. the sequel follows the master criminals as they try to pull off three major heists across europe. it knocked last week s number one national treasure into third place. wesley snipes blade: trinity was in second taking $16.1m (£8...


In [12]:
# Word2Vec
search_w2v(query_sentence) 

,ArticleId,Text
1175,1117,stormy year for property insurers a string of storms typhoons and earthquakes has made 2004 the most expensive year on record for property insurers according to swiss re. the world s second biggest insurer said disasters around the globe have seen property claims reach $42bn (£21.5bn). 2004 reinforces the trend towards higher losses said swiss re. tightly packed populations in the areas involved in natural and man-made disasters were to partly to blame for the rise in claims it said. ...
1017,1790,us tv special for tsunami relief a us television network will screen a celebrity tv special to benefit the tsunami relief effort in south asia. nbc will encourage viewer donations during an hour-long show featuring musical performances on 15 january. actress sandra bullock has donated $1m (£525 000) to the american red cross and actor leonardo dicaprio pledged a sizable aid contribution to unicef. meanwhile 70 hong kong music and movie stars re-recorded we are the world in mandarin and ca...
2133,243,china now top trader with japan china overtook the us to become japan s biggest trading partner in 2004 according to numbers released by japan s finance ministry on wednesday. china accounted for 20.1% of japan s trade in 2004 compared with 18.6% for the us. in 2003 the us was ahead with 20.5% and china came second with 19.2%. the change highlights china s growing importance as an economic powerhouse. in 2004 japan s imports from and exports to china (and hong kong) added up to 22 201bn...
677,766,lufthansa flies back to profit german airline lufthansa has returned to profit in 2004 after posting huge losses in 2003. in a preliminary report the airline announced net profits of 400m euros ($527.61m; £274.73m) compared with a loss of 984m euros in 2003. operating profits were at 380m euros ten times more than in 2003. lufthansa was hit in 2003 by tough competition and a dip in demand following the iraq war and the killer sars virus. it was also hit by troubles at its us catering bus...
2071,837,tsunami cost hits jakarta shares the stock market in jakarta has seen its biggest slide in a month after the country doubled the likely cost of rebuilding from the asian tsunami. the fall came as indonesia said it expected debt repayments of up to 30 trillion rupiah ($3.2bn; £1.7bn) to be frozen to help pay for the recovery. by monday s close the jakarta stock exchange was down 2.1% at 1 011.15. bar a slight dip at the new year the jse had risen steadily by 4.7% since the tsunami hit on ...


In [13]:
# SentenceTransformers
search_transformers(query_sentence)

ArticleId                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   1485
Text         tsunami to cost sri lanka $1.3bn sri lanka faces a $1.3bn (£691m) bill in 2005 for reconstruction after the tsunami which killed more than 30 000 of its people  its central bank says.  this estimate is preliminary  bank governor sunil mendis told reporters  and could rise in 2006. the island state is asking for about $320m from the international monetary fund to help pay for relief  he said. the bank has 5bn rupees ($50m; £27m) set aside to lend at a lower interest rate

# Conclusion

We built semantic search engines using TF-IDF, Word2Vec, and Sentence Transformers to retrieve the top-5 closest news from the dataset that match this query sentence. It seems that the transformer model performs best while it takes a longer time. However, picking the best model can be subjective, so A/B tests are needed based on click-through rates and session durations.